### imports

In [1]:
import os
import json
import yaml
import copy
import torch
import shutil
import splitfolders
from copy import deepcopy
from model import *
from ultralytics import YOLO

### Cloning of TACO repository and environment

In [2]:
if not os.path.exists(os.path.join(os.path.dirname(os.getcwd()), 'TACO')):
    !git clone https://github.com/pedropro/TACO
    %cd TACO
    !python download.py
    %cd ..
    !pip install -r ./TACO/requirements.txt
else:
    print("TACO already cloned to ./TACO directory.")

TACO already cloned to ./TACO directory.


### Creation of images and labels folders to store data in YOLO formatting

In [3]:
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO"))
else:
    print("TACO dataset folder already exists.")
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/images")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/images"))
else:
    print("TACO images folder already exists.")
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/labels")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/labels"))
else:
    print("TACO labels folder already exists.")

TACO dataset folder already exists.
TACO images folder already exists.
TACO labels folder already exists.


In [4]:
main_folder = os.path.join(os.path.dirname(os.getcwd()), "TACO/data")
new_folder = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/images")
if os.listdir(new_folder):
    print("TACO images already in the destination folder.")
else:
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.jpg', '.jpeg')):
                    new_filename = f"{subfolder}_{filename}"
                    src_path = os.path.join(subfolder_path, filename)
                    dest_path = os.path.join(new_folder, new_filename)
                    shutil.copy(src_path, dest_path)
    print("Images were copied successfully .")

TACO images already in the destination folder.


### Converting COCO format to YOLO using annotations.json info

In [5]:
annotaions_file = os.path.join(os.path.dirname(os.getcwd()), 'TACO/data/annotations.json')
with open(annotaions_file, 'r') as f:
    annotations = json.load(f)

In [6]:
original_to_custom_categories = {
    'bio': ["Food waste"],
    'glass': ['Broken glass', 'Glass bottle', 'Glass jar', 'Glass cup'],
    'metals_and_plastics': ['Aerosol', 'Aluminium foil', 'Clear plastic bottle',
                'Disposable food container', 'Disposable plastic cup', 'Drink can',
                'Drink carton', 'Food Can', 'Food can', 'Metal bottle cap', 'Metal lid',
                'Other plastic', 'Other plastic bottle', 'Other plastic container',
                'Other plastic cup', 'Plastic Film', 'Plastic bottle cap', 'Plastic film',
                'Plastic glooves', 'Plastic lid', 'Plastic straw', 'Plastic utensils',
                'Polypropylene bag', 'Pop tab', 'Scrap metal', 'Single-use carrier bag',
                'Six pack rings', 'Spread tub', 'Tupperware'],
    'non_recyclable': ['Aluminium blister pack', 'Carded blister pack', 'Cigarette',
                'Crisp packet', 'Foam cup', 'Foam food container', 'Garbage bag',
                'Meal carton', 'Meal carton', 'Other plastic wrapper', 'Paper cup',
                'Paper straw', 'Pizza box', 'Plastified paper bag', 'Rope', 'Rope & strings',
                'Shoe', 'Squeezable tube', 'Styrofoam piece', 'Tissues', 'Wrapping paper'],
    'other': ["Battery"],
    'paper': ["Corrugated carton","Egg carton","Toilet tube","Other carton",
                'Magazine paper', "Normal paper", "Paper bag"],
    'unknown': ["Unlabeled litter"]
}
original_to_custom_reverse = {}
for custom, originals in original_to_custom_categories.items():
    for original in originals:
        original_to_custom_reverse[original] = custom

In [7]:
labels_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO/labels")
if os.listdir(labels_dir):
    print("Annotations per image already saved in YOLO format under image file name")
else:
    for image_info in annotations['images']:
        image_id = image_info['id']
        image_file_name = image_info['file_name'].replace('/', '_').split('.')[0] + '.txt'
        file_path = os.path.join(labels_dir, image_file_name)
        width, height = image_info['width'], image_info['height']
        annotation_data = [ann for ann in annotations['annotations'] if ann['image_id'] == image_id]
        buffer = ''
        for annotation in annotation_data:
            bbox = annotation['bbox']
            if bbox[2] < 1 or bbox[3] < 1:
                continue
            x_center = (bbox[0] + bbox[2] / 2) / width
            y_center = (bbox[1] + bbox[3] / 2) / height
            w = bbox[2] / width
            h = bbox[3] / height
            original_category_name = annotations['categories'][annotation['category_id'] - 1]['name']
            custom_category = original_to_custom_reverse.get(original_category_name, 'unknown')
            new_category_id = custom_to_new_id[custom_category]
            buffer += f"{new_category_id} {x_center} {y_center} {w} {h}\n"
        with open(file_path, 'w') as f:
            f.write(buffer)

Annotations per image already saved in YOLO format under image file name


### Splitting the data into train, validation, and test sets

In [8]:
data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO")
data_path_split = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO_DATASET")
if not os.path.exists(data_path_split):
    splitfolders.ratio(data_path, output=data_path_split, seed=0, ratio=(.8, .1, .1))
else:
    print("TACO dataset already split.")

TACO dataset already split.


### Creating yaml file for YOLO train function

In [9]:
yaml_path = os.path.join(os.getcwd(), 'config.yaml')
if os.path.exists(yaml_path):
    print(f"{yaml_path} already exists.")
else:
    yaml_dict = {
        'train': os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO_DATASET/train/images"),
        'val': os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO_DATASET/val/images"),
        'test': os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "imgs/TACO_DATASET/test/images"),
        'names': custom_categories
    }
    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(yaml_dict, yaml_file)

/home/hay.e/DeepLearning/Project/mini_project/project/YOLOv8/config.yaml already exists.


### Train and Validation - Optimizers experiment

In [10]:
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/Optimizers_exp")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/Optimizers_exp"))
else:
    print("Optimizers experiment results folder already exists.")

Optimizers experiment results folder already exists.


In [21]:
model_template = initialize_model()
optimizers = ['SGD', 'Adam', 'AdamW', 'RMSProp']
for optimizer in optimizers:
    model = copy.deepcopy(model_template)
    folder_name = f"{optimizer}"
    save_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), f'results/Optimizers_exp/{folder_name}')
    if not os.path.exists(save_dir):
        extra_params = {"pretrained": True, "optimizer": optimizer, "epochs": 100}
        train_custom_model(model, save_dir, yaml_path, extra_params)
    else:
        print(f'{folder_name} experiment already done and results were saved at {save_dir}')

SGD experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/Optimizers_exp/SGD
Adam experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/Optimizers_exp/Adam
AdamW experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/Optimizers_exp/AdamW
RMSProp experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/Optimizers_exp/RMSProp


### Train and Validation - HyperParameters tuning experiment

In [12]:
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/HyperParams_tuning_exp")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/HyperParams_tuning_exp"))
else:
    print("Hyper parameters tuning experiment results folder already exists.")

Hyper parameters tuning experiment results folder already exists.


In [22]:
model_template = initialize_model()

learning_rates = [0.01, 0.015]
momentums = [0.9, 0.937, 0.97]
batch_sizes = [4, 8, 16]

for lr in learning_rates:
    for momentum in momentums:
        for batch_size in batch_sizes:
            model = copy.deepcopy(model_template)
            folder_name = f"lr_{lr}_momentum_{momentum}_batch_size_{batch_size}"
            save_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), f'results/HyperParams_tuning_exp/{folder_name}')
            if not os.path.exists(save_dir):
                extra_params = {
                    "pretrained": True,
                    "optimizer": "SGD",
                    "epochs": 70,
                    "lr0": lr,
                    "momentum": momentum,
                    "batch": batch_size,
                    "lrf": lr/10
                }
                print(f"Training with {extra_params}")
                train_custom_model(model, save_dir, yaml_path, extra_params)
            else:
                print(f'{folder_name} experiment already done and results were saved at {save_dir}')

lr_0.01_momentum_0.9_batch_size_4 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/HyperParams_tuning_exp/lr_0.01_momentum_0.9_batch_size_4
lr_0.01_momentum_0.9_batch_size_8 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/HyperParams_tuning_exp/lr_0.01_momentum_0.9_batch_size_8
lr_0.01_momentum_0.9_batch_size_16 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/HyperParams_tuning_exp/lr_0.01_momentum_0.9_batch_size_16
lr_0.01_momentum_0.937_batch_size_4 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/HyperParams_tuning_exp/lr_0.01_momentum_0.937_batch_size_4
lr_0.01_momentum_0.937_batch_size_8 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/HyperParams_tuning_exp/lr_0.01_momentum_0.937_batch_size_8
lr_0.01_momentu

### Train and Validation - Freezing Layers experiment

In [14]:
if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/Freezing_layers_exp")):
    os.makedirs(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "results/Freezing_layers_exp"))
else:
    print("Freezing layers experiment results folder already exists.")

Freezing layers experiment results folder already exists.


In [23]:
model = initialize_model()
model.add_callback("on_train_start", freeze_layer)
folder_name = f"frozen_yolo_SGD_lr0_0.001_momentum_0.97_batch_8"
save_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), f'results/Freezing_layers_exp/{folder_name}')
if not os.path.exists(save_dir):
    extra_params = {
        "pretrained": True,
        "optimizer": "SGD",
        "epochs": 100,
        "lr0": 0.001,
        "momentum": 0.97,
        "batch": 8,
        "lrf": 0.0001
    }
    train_custom_model(model, save_dir, yaml_path, extra_params)
else:
    print(f'{folder_name} experiment already done and results were saved at {save_dir}')

frozen_yolo_SGD_lr0_0.001_momentum_0.97_batch_8 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/results/Freezing_layers_exp/frozen_yolo_SGD_lr0_0.001_momentum_0.97_batch_8


### Testing

In [26]:
chosen_weights_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'results/HyperParams_tuning_exp/lr_0.001_momentum_0.97_batch_size_82/weights/best.pt')
save_dir_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'tests')
if not os.listdir(save_dir_path):
    model = YOLO(chosen_weights_path)
    eval_results = evaluate_model_performance(model, save_dir_path)
else:
    print(f'Testing already done and results were saved at {save_dir_path}')

Testing already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests


### Testing - IoU thresholds experiment

In [29]:
chosen_weights_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'results/HyperParams_tuning_exp/lr_0.001_momentum_0.97_batch_size_82/weights/best.pt')
save_dir_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'tests')
model_template = YOLO(chosen_weights_path)
for iou in [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]:
    if not os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), f'tests/IoU_threshold_{iou}')):
        model = copy.deepcopy(model_template)
        eval_results = evaluate_model_performance(model, save_dir_path, iou)
    else:
        print(f'IoU threshold {iou} experiment already done and results were saved at {save_dir_path}/IoU_threshold_{iou}')

IoU threshold 0.3 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.3
IoU threshold 0.4 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.4
IoU threshold 0.5 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.5
IoU threshold 0.7 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.7
IoU threshold 0.8 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.8
IoU threshold 0.9 experiment already done and results were saved at /home/hay.e/DeepLearning/Project/mini_project/tests/IoU_threshold_0.9
